In [7]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as fun
from pyspark.sql.types import *
import pyarrow as pa
import pyarrow.parquet as pq
%matplotlib inline
import matplotlib.pyplot as plt
from pyspark.sql.functions import udf
import os
import snscrape.modules.twitter as sntwitter
import pandas as pd
spark_conf = SparkConf()
spark_conf = spark_conf.setAppName('spark-nlp-book-p1c1')
spark_conf = spark_conf.setAppName('master[*]')
packages = ','.join([
    "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.4"
])

spark_conf = spark_conf.set("spark.jars.packages", packages)
spark_conf.set('spark.executor.memory', '8G')
spark_conf.set('spark.driver.memory', '4G')
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

In [2]:
all_files = os.listdir("/Users/keithwhitson/Projekte/russian-troll-tweets/")
all_files = filter(lambda x: 'csv' in x, all_files)
all_files = filter(lambda x: 'russia' not in x, all_files)
all_files = list(all_files)

In [3]:
df = pd.concat(map(lambda file: pd.read_csv(file, low_memory=False), all_files))
df["external_author_id"] = df["external_author_id"].astype(str)
df['alt_external_id'] = df['alt_external_id'].astype(str)
df['tweet_id'] = df['tweet_id'].astype(str)
df['tco1_step1'] = df['tweet_id'].astype(str)
df['tco2_step1'] = df['tweet_id'].astype(str)
df['region'] = df['region'].astype(str)
df['tco3_step1'] = df['tco3_step1'].astype(str)
df['post_type'] = df['post_type'].astype(str)
df['content'] = df['content'].astype(str)

In [4]:
df.to_parquet('russian_tweets_from_df.parquet',compression="snappy",partition_cols=["author"])

In [8]:
sdf = spark.read.csv(all_files,inferSchema=True,header=True)
#sdf.write.parquet(path='russian_tweets_from_sdf.parquet',partitionBy='author',compression='snappy')
#sdf.write.parquet(path='s3://twittermachinelearningstuff/russian_tweets_from_sdf.parquet',partitionBy='author',compression='snappy')

Py4JJavaError: An error occurred while calling o143.parquet.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileFormat(DataSource.scala:461)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWriting(DataSource.scala:556)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:382)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:355)
	at or